# Задачи
### 1. Оценить количество ботов, которые заходят на наш сайт, и их долю от всего трафика.
### 2. Определить, с каких источников и в какое время суток чаще всего заходят роботы.

In [ ]:
#Загрузка необходимых библиотек
import json 
import requests
from pprint import pprint
import pandas as pd

In [ ]:
params = {'metrics': 'ym:s:visits,ym:s:robotPercentage',
          'dimensions':'ym:s:lastsignTrafficSource,ym:s:hour',
          'date1': '2020-09-01',  
          'date2': '2020-09-30',     
          'ids': 30177909,
          'accuracy':'full',
          'limit':100000}

token = 'AQAAAAA3MYDsAAdYqajdxw9RJkvim0w3F0XJB00'
headers = {'Authorization': 'OAuth ' + token}

In [ ]:
#Запрос к Яндекс Метрике 
response = requests.get('https://api-metrika.yandex.net/stat/v1/data', params=params, headers=headers)
print (response.status_code) #код ответа, полученный от API

200


In [ ]:
#Переводим ответ от API в словарь
metrika_data = response.json()
print(type(metrika_data))

<class 'dict'>


In [ ]:
#Количество строк, которые соответствуют нашему запросу:
metrika_data['total_rows']

107

In [ ]:
#Выведем значение по ключу data в словаре metrika_dat
pprint(metrika_data['data'][0:5])

[{'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '16', 'name': '16:00'}],
  'metrics': [251.0, 24.70119522]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '15', 'name': '15:00'}],
  'metrics': [240.0, 26.25]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '11', 'name': '11:00'}],
  'metrics': [228.0, 25.87719298]},
 {'dimensions': [{'icon_id': '2',
                  'icon_type': 'traffic-source',
                  'id': 'organic',
                  'name': 'Search engine traffic'},
                 {'id': '13', 'name': '13:00'}],
  'metrics': [227.0, 2

In [ ]:
#Cоздаем датафрейм
metrika_df = pd.DataFrame(metrika_data['data'])
display(metrika_df.head()) 

,dimensions,metrics
0,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[251.0, 24.70119522]"
1,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[240.0, 26.25]"
2,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[228.0, 25.87719298]"
3,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[227.0, 28.1938326]"
4,"[{'icon_id': '2', 'icon_type': 'traffic-source...","[225.0, 24.88888889]"


In [ ]:
#Преобразуем данные в так называемый плоский вид, то есть избавимся от вложенных словарей
def getMetrikaDataInListOfDicts(metrika_data):
    list_of_dicts = []
    dimensions_list = metrika_data['query']['dimensions']
    metrics_list = metrika_data['query']['metrics']
    for data_item in metrika_data['data']:
        d = {}
        for i,dimension in enumerate(data_item['dimensions']):
            d[dimensions_list[i]] = dimension['name']
        for i,metric in enumerate(data_item['metrics']):
            d[metrics_list[i]] = metric
        list_of_dicts.append(d)
    return list_of_dicts

In [ ]:
metrika_list_of_dicts = getMetrikaDataInListOfDicts(metrika_data)
pprint(metrika_list_of_dicts[0:3])

[{'ym:s:hour': '16:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 24.70119522,
  'ym:s:visits': 251.0},
 {'ym:s:hour': '15:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 26.25,
  'ym:s:visits': 240.0},
 {'ym:s:hour': '11:00',
  'ym:s:lastsignTrafficSource': 'Search engine traffic',
  'ym:s:robotPercentage': 25.87719298,
  'ym:s:visits': 228.0}]


In [ ]:
#Создаем итоговый датафрейм
metrika_df = pd.DataFrame(metrika_list_of_dicts)
display(metrika_df.head())

,ym:s:hour,ym:s:lastsignTrafficSource,ym:s:robotPercentage,ym:s:visits
0,16:00,Search engine traffic,24.701195,251.0
1,15:00,Search engine traffic,26.250000,240.0
2,11:00,Search engine traffic,25.877193,228.0
3,13:00,Search engine traffic,28.193833,227.0
4,12:00,Search engine traffic,24.888889,225.0


In [ ]:
#Меняем названия столбцов и создаем столбец с количеством ботов
metrika_df.columns=['Time','Source','RobotPercentage', 'Visits']
metrika_df['RobotCount'] = metrika_df['Visits']*metrika_df['RobotPercentage']/100
display(metrika_df.head())

,Time,Source,RobotPercentage,Visits,RobotCount
0,16:00,Search engine traffic,24.701195,251.0,62.0
1,15:00,Search engine traffic,26.250000,240.0,63.0
2,11:00,Search engine traffic,25.877193,228.0,59.0
3,13:00,Search engine traffic,28.193833,227.0,64.0
4,12:00,Search engine traffic,24.888889,225.0,56.0


In [ ]:
#Считаем общую долю ботов
a=metrika_df.RobotCount.sum()
b=metrika_df.Visits.sum()
c=a/b*100
print('На сайт заходят {} ботов, их доля от всего трафика составляет {}%'.format(round(a,0),round(c,2)))

На сайт заходят 962.0 ботов, их доля от всего трафика составляет 22.29%


In [ ]:
#Источники, с которых заходят роботы
metrika_df.groupby(['Source']).RobotCount.sum().sort_values(ascending = False)

Source
Search engine traffic            739.0
Direct traffic                   177.0
Link traffic                      31.0
Social network traffic            11.0
Internal traffic                   3.0
Cached page traffic                1.0
Recommendation system traffic      0.0
Name: RobotCount, dtype: float64

In [ ]:
#Время, в которое чаще заходят роботы
metrika_df.groupby(['Time']).RobotCount.sum().sort_values(ascending = False).head(7)

Time
13:00    80.0
16:00    75.0
12:00    75.0
14:00    74.0
15:00    71.0
11:00    70.0
17:00    66.0
Name: RobotCount, dtype: float64

# Выводы
### 1. На сайт заходят 962.0 ботов, их доля от всего трафика составляет 22.29%
### 2. Боты чаще всего заходят с Search engine traffic и днём с 12 до 16 часов